# Duration Prediction

In [5]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import root_mean_squared_error
import pickle
import mlflow
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [6]:
pd.options.mode.copy_on_write = True

In [7]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")
mlflow.autolog(disable=True)

<Experiment: artifact_location='/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1748211062137, experiment_id='1', last_update_time=1748211062137, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [8]:
def preprocessing(df):
    # compute target variables: trip duration in minutes
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    # keep only trips of less than an hour
    df = df.loc[(df.duration >= 1) & (df.duration <= 60), :]

    # extract trip date components
    df['trip_month'] = pd.Categorical(df.lpep_pickup_datetime.dt.month)
    df['trip_dom'] = pd.Categorical(df.lpep_pickup_datetime.dt.day)
    df['trip_hour'] = pd.Categorical(df.lpep_pickup_datetime.dt.hour)

    # concatenate pickup and dropoff locations
    df['PU_DO'] = df['PULocationID'].astype(str) + '_' + df['DOLocationID'].astype(str)

    return df

In [9]:
def one_hot_encoding(df, numerical, categorical, dv=None):
    df[categorical] = df[categorical].astype(str)
    df_dicts = df[categorical + numerical].to_dict(orient='records')

    if not dv:
        dv = DictVectorizer()
        dv.fit(df_dicts)

    X_train = dv.transform(df_dicts)

    return X_train, dv

In [10]:
def train_model(model, X_train, y_train):
    model.fit(X_train, y_train)

In [11]:
# read im the raw data
df_train = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

In [12]:
# data cleaning
df_train = preprocessing(df_train)
df_val = preprocessing(df_val)

In [13]:
# variable selection
categorical = ['PU_DO']
numerical = ['trip_distance']

In [14]:
# categorical feature encoding
X_train, dv = one_hot_encoding(df_train, numerical, categorical)
X_val, _ = one_hot_encoding(df_val, numerical, categorical, dv=dv)

In [15]:
target = 'duration'
y_train = df_train[target]
y_val = df_val[target]

### Linear Regression

In [16]:
lr = LinearRegression()
train_model(lr, X_train, y_train)

In [17]:
y_pred = lr.predict(X_val)

In [18]:
root_mean_squared_error(y_val, y_pred)

7.758715200888857

### Lasso

In [19]:
alpha = .1
lasso = Lasso(alpha=alpha)
train_model(lasso, X_train, y_train)
y_pred = lasso.predict(X_val)
rmse = root_mean_squared_error(y_val, y_pred)

with mlflow.start_run():
    mlflow.set_tag("developer", "Armand Winant")

    # datasets used for training and validation
    mlflow.log_param("train-data-path", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet")
    mlflow.log_param("validation-data-path", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet")

    # hyperparameters
    mlflow.log_param("alpha", alpha)

    # validation performance
    mlflow.log_metric("rmse", rmse)

### XGBoost
#### Manual logging

In [ ]:
mlflow.xgboost.autolog(disable=True)

In [20]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [21]:
def objective(params):
    booster = xgb.train(
        params=params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, "validation")],
        early_stopping_rounds=50
    )
    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)

    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}


In [22]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:squarederror',
    'seed': 42
}

In [23]:
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:10:14] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:10.94950                          
[1]	validation-rmse:9.95155                           
[2]	validation-rmse:9.17225                           
[3]	validation-rmse:8.57192                           
[4]	validation-rmse:8.11506                           
[5]	validation-rmse:7.76265                           
[6]	validation-rmse:7.49885                           
[7]	validation-rmse:7.29836                           
[8]	validation-rmse:7.14703                           
[9]	validation-rmse:7.03180                           
[10]	validation-rmse:6.94461                          
[11]	validation-rmse:6.87548                          
[12]	validation-rmse:6.82452                          
[13]	validation-rmse:6.78113                          
[14]	validation-rmse:6.74823                          
[15]	validation-rmse:6.72440                          
[16]	validation-rmse:6.70538                          
[17]	validation-rmse:6.68984                          
[18]	valid

/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:11:28] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.65561                                                      
[1]	validation-rmse:6.81852                                                      
[2]	validation-rmse:6.65147                                                      
[3]	validation-rmse:6.59759                                                      
[4]	validation-rmse:6.57294                                                      
[5]	validation-rmse:6.56654                                                      
[6]	validation-rmse:6.56133                                                      
[7]	validation-rmse:6.55555                                                      
[8]	validation-rmse:6.54920                                                      
[9]	validation-rmse:6.54374                                                      
[10]	validation-rmse:6.53626                                                     
[11]	validation-rmse:6.53216                                                     
[12]	validation-

...

/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:56:41] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:11.14419                                                     
[1]	validation-rmse:10.26022                                                     
[2]	validation-rmse:9.53455                                                      
[3]	validation-rmse:8.93885                                                      
[4]	validation-rmse:8.45850                                                      
[5]	validation-rmse:8.06908                                                      
[6]	validation-rmse:7.75858                                                      
[7]	validation-rmse:7.51223                                                      
[8]	validation-rmse:7.31463                                                      
[9]	validation-rmse:7.15634                                                      
[10]	validation-rmse:7.02947                                                     
[11]	validation-rmse:6.92854                                                     
[12]	validation-

/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:58:16] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:11.49788                                                    
[1]	validation-rmse:10.86659                                                    
[2]	validation-rmse:10.30877                                                    
[3]	validation-rmse:9.82021                                                     
[4]	validation-rmse:9.39232                                                     
[5]	validation-rmse:9.02131                                                     
[6]	validation-rmse:8.69100                                                     
[7]	validation-rmse:8.40905                                                     
[8]	validation-rmse:8.16371                                                     
[9]	validation-rmse:7.95457                                                     
[10]	validation-rmse:7.76748                                                    
[11]	validation-rmse:7.60965                                                    
[12]	validation-rmse:7.47300

KeyboardInterrupt: 

#### Automatic logging

In [ ]:
mlflow.xgboost.autolog()

In [ ]:
params = {
    'learning_rate': 0,
    'max_depth': 0,
    'min_child_weight': 0,
    'objective': 'reg:squarederror',
    'reg_alpha': 0,
    'reg_lambda': 0,
    'seed': 42
}

booster = xgb.train(
        params=params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, "validation")],
        early_stopping_rounds=50
    )

y_pred = booster.predict(valid)
rmse = root_mean_squared_error(y_val, y_pred)